# **Homework 8 - Anomaly Detection**

If there are any questions, please contact mlta-2022spring-ta@googlegroups.com

Slide:    [Link]()　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw8)

# Set up the environment


## Package installation

## Downloading data

# Import packages

In [1]:
firstTime = False
ensemblePart = 1
coderType = 0
import torch
torch.cuda.empty_cache()
!nvidia-smi

Sun May  8 03:41:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
if firstTime:
  !pip install -q qqdm
  !wget https://github.com/MachineLearningHW/HW8_Dataset/releases/download/v1.0.0/data.zip
  !unzip data.zip
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
from qqdm import qqdm, format_str
import pandas as pd

# Loading data

In [3]:

train = np.load('data/trainingset.npy', allow_pickle=True)
test = np.load('data/testingset.npy', allow_pickle=True)

print(train.shape)
print(test.shape)

(100000, 64, 64, 3)
(19636, 64, 64, 3)


## Random seed
Set the random seed to a certain value for reproducibility.

In [4]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(48763)

# Autoencoder

# Models & loss

In [5]:
from random import randint
import tensorflow as tf 
class all(nn.Module) :
    def contracting_block(self, in_channels, out_channels, ker_size):
        block = torch.nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, ker_size, stride=2, padding=1),
                    nn.ReLU(),
                    nn.BatchNorm2d(out_channels),
                    nn.Conv2d(out_channels, out_channels, ker_size),
                    nn.ReLU(),
                    nn.BatchNorm2d(out_channels),
                )
        return block

    def __init__(self):
        super(all, self).__init__()
        self.t1_encoder = nn.Sequential(
            nn.Linear(64 * 64 * 3, 1024*3),
            nn.ReLU(),
            nn.Linear(1024*3, 256*3),
            nn.ReLU(),
            nn.Linear(256*3, 64*3),
            nn.ReLU(),
            nn.Linear(64*3, 16*3),
            nn.ReLU(), 
            nn.Linear(16*3, 8*3),
        )    
        self.t3_encoder = nn.Sequential(
            self.contracting_block(3, 6, 2),
            nn.MaxPool2d(kernel_size=2),
            self.contracting_block(6, 12, 2),
            nn.MaxPool2d(kernel_size=2),
            self.contracting_block(12, 24, 2),
            nn.MaxPool2d(kernel_size=2),   
        )
        self.t5_encoder = nn.Sequential(
            nn.Conv2d(3, 28, kernel_size=4, stride=2),    #(64-3)/2=31
            nn.PReLU(),  
            nn.MaxPool2d(kernel_size=2, stride=2),        #(31-1)/2=15
            nn.Conv2d(28, 48, kernel_size=2, stride=1),   #(15-1)/1=14
            nn.PReLU(),  
            nn.MaxPool2d(kernel_size=2, stride=2),        #(14-1)/2=7
            nn.Conv2d(48, 64, kernel_size=2, stride=1),   #(7-1)/1=6
            nn.PReLU()  
        )
        self.t5_linear = nn.Sequential(
            nn.Linear(64 * 6 * 6, 256), 
            nn.PReLU(),
            nn.Linear(256, 24),
        )
        self.t1_decoder = nn.Sequential(
            nn.Linear(8*3, 16*3),
            nn.ReLU(),
            nn.Linear(16*3, 64*3),
            nn.ReLU(), 
            nn.Linear(64*3, 256*3),
            nn.ReLU(), 
            nn.Linear(256*3, 1024*3),
            nn.ReLU(), 
            nn.Linear(1024*3, 4096*3),
            nn.Tanh()
        )


    def forward(self, img):
        imgf = img.view(img.shape[0], -1)
        img1 = self.t1_encoder(imgf)
        global ensemblePart

        if ensemblePart == 0:
          img3 = self.t3_encoder(img)
          img3 = img3[:,:,0,0]
          x = (img1+img3)/2
          x = self.t1_decoder(x)

        elif ensemblePart == 1:
          img5 = self.t5_encoder(img)
          img5 = img5.view(img5.shape[0], img5.shape[1]*img5.shape[2]*img5.shape[3])
          img5 = self.t5_linear(img5)
          x = (img1+img5)/2
          x = self.t1_decoder(x)
        return x          

# Dataset module

Module for obtaining and processing data. The transform function here normalizes image's pixels from [0, 255] to [-1.0, 1.0].


In [6]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)
        
        self.transform = transforms.Compose([
          transforms.Lambda(lambda x: x.to(torch.float32)),
          transforms.Lambda(lambda x: 2. * x/255. - 1.),
        ])
        
    def __getitem__(self, index):
        x = self.tensors[index]
        
        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

# Training

## Configuration


In [7]:
# Training hyperparameters
num_epochs = 10#100
batch_size = 2000
learning_rate = 1e-3

# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Model
model_type = 'all'   # selecting a model type from {'cnn', 'fcn', 'vae', 'resnet'}
model_classes = {'all': all()}
model = model_classes[model_type].cuda()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6, eta_min=0, last_epoch=-1)

## Training loop

In [8]:
best_loss = np.inf
model.train()

qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
for epoch in qqdm_train:
    tot_loss = list()
    for data in train_dataloader:
        img = data.float().cuda()

        output = model(img)
        if model_type in ['all'] and coderType == 0:
            img = img.view(img.shape[0], -1)
        
        loss = criterion(output, img)
        tot_loss.append(loss.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #scheduler.step()
    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
    # ===================log========================
    qqdm_train.set_infos({
        'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
        'loss': f'{mean_loss:.4f}',
    })
    # ===================save_last========================
    torch.save(model, 'last_model_{}.pt'.format(model_type))

 Iters    Elapsed Time      Speed                                               
 0/10           -             -                                                 
Description   0.0% |                                                           | Iters    Elapsed Time      Speed    epoch   loss                               
 1/10   00:00:18<00:02:49  0.05it/s  1/10   0.2190                              
Description  10.0% |█████                                                      | Iters    Elapsed Time      Speed    epoch   loss                               
 2/10   00:00:37<00:02:31  0.05it/s  2/10   0.1607                              
Description  20.0% |███████████                                                | Iters    Elapsed Time      Speed    epoch   loss                               
 3/10   00:00:56<00:02:11  0.05it/s  3/10   0.1239                              
Description  30.0% |█████████████████                                          | Iters    Elapsed Time      Spee

# Inference
Model is loaded and generates its anomaly score predictions.

## Initialize
- dataloader
- model
- prediction file

In [9]:
eval_batch_size = 200

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=1)
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = f'last_model_{model_type}.pt'
model = torch.load(checkpoint_path)
model.eval()

# prediction file 
out_file = 'prediction_'+str(ensemblePart)+'.csv'

In [10]:
anomality = list()
with torch.no_grad():
  for i, data in enumerate(test_dataloader):
    img = data.float().cuda()
    output = model(img)
    
    if model_type in ['all'] and coderType == 0:     
      img = img[:,:,12:56,12:56]
      img = img.contiguous().view(img.shape[0], img.shape[1]*img.shape[2]*img.shape[3])
      a,b = output.shape
      output = output.view(a, b//(64*64), 64, 64)
      output = output[:,:,12:56,12:56]
      output = output.contiguous().view(output.shape[0], -1)
      loss = eval_loss(output, img).sum(-1)

    elif model_type in ['all'] and coderType == 1:
      img = img[:,:,12:56,12:56]
      output = output[:,:,12:56,12:56]
      loss = eval_loss(output, img).sum([1, 2, 3])

    anomality.append(loss)
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()

df = pd.DataFrame(anomality, columns=['score'])
df.to_csv(out_file, index_label = 'ID')

In [11]:
####################
###   ensemble   ###
### average csv. ###
####################
import csv
with open('prediction_0.csv', newline='') as csvfile:
  rows = list(csv.reader(csvfile))
  #print(rows[0])
  #print(rows[1])
  csvfile.close()
with open('prediction_1.csv', newline='') as csvfile:
  table = list(csv.reader(csvfile))
  #print(table[0])
  #print(table[1])
  for i in range(1,len(table)):
    table[i][1] = (float(table[i][1])+float(rows[i][1]))/2
    #print(table[i][1])
  csvfile.close()

csvfile = open('prediction.csv', 'w')
writer = csv.writer(csvfile)
writer.writerows(table)
csvfile.close()

In [12]:
data = test_dataloader[2]
data = np.transpose(data, (1, 2, 0))
from matplotlib import pyplot as plt
plt.imshow(data[12:56,12:56,:], aspect="auto")
#plt.show()
plt.savefig('origin.png')

#output = model(img)

res = output[2].cpu().detach().numpy()
print(res.shape)
res = np.reshape(res, [3,44,44])
res = np.transpose(res, (1,2,0))
from matplotlib import pyplot as plt
plt.imshow(res[12:56,12:56,:], aspect="auto")
#plt.show()
plt.savefig('final.png')

TypeError: ignored